In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def entry_transform(number,N):
    i = int(number/N)
    j = number-i*N
    return i,j

In [3]:
def create_lattice(N,particle_number): 
    #create the empty lattice
    lattice_matrix = np.zeros(N*N).reshape((N,N))
    
    #fill the lattice with particle_number particles
    part_positions = np.random.choice(a = np.arange(N*N), size = particle_number, replace = False)
    
    #now transform the particle positions to fill in the lattice matrix
    for k in range(particle_number):
        i,j = entry_transform(part_positions[k],N)
        lattice_matrix[i,j] = 1 #place particle in the correct position
    
    return lattice_matrix

In [4]:
def create_neighs():
    even_neighbours = [[0,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]
    odd_neighbours = [[0,1],[1,0],[1,1],[0,-1],[-1,0],[-1,1]]
    neighs = [even_neighbours,odd_neighbours]  
    return neighs

In [5]:
def int_energy(lattice,N,i,j):
    #calculate the energy of the current configuration by counting the number of neighbouring particles
    neighs = create_neighs()
    
    energy = 0
    #iterate through the neighbours
    for neigh in neighs[i%2]:
        neigh_i, neigh_j = (i+neigh[0])%N, (j+neigh[1])%N
        energy += lattice[neigh_i,neigh_j]
    return energy

In [6]:
def new_configurations(lattice, N, Beta):
    neighs = create_neighs()
    
    new_configs = []
    rates = []
    #iterate through the lattice
    for i in range(N):
        for j in range(N):
            if lattice[i,j] == 1: #there is a particle in the lattice
                E_old = int_energy(lattice,N,i,j) #calculate the energy of the old configuration
                for neigh in neighs[i%2]: #iterate through the neighbours
                    neigh_i, neigh_j = (i+neigh[0])%N, (j+neigh[1])%N #position of the neighbouring site in the lattice
                    #then if the neighbour is empty we can hop into it
                    if lattice[neigh_i,neigh_j] == 0:
                        #calculate the energy for this empty neighbouring site
                        E_new = int_energy(lattice,N,neigh_i,neigh_j)-1
                        rate = np.exp(-(E_old-E_new)*Beta)
                        new_configs.append([i,j,neigh_i,neigh_j,E_old,E_new])
                        rates.append(rate)
    return new_configs, rates

In [7]:
def update_lattice(lattice, i, j, new_i, new_j):
    #destroy particle at site i,j
    lattice[i, j] = 0
    
    #and recreate the particle at site new_i,new_j
    lattice[new_i, new_j] = 1
    
    #return the lattice
    return lattice

In [8]:
def find_transition(lattice, N, new_configs, rates, Beta):
    #from the possible new configurations randomly choose one and carry it out
    
    #total rate/"partition function"
    tot_rate = sum(rates)
    
    u = np.random.random()
    val = u*tot_rate
    
    l = -1
    while val > 0:
        l += 1
        val -= rates[l]
    #we need to carry out the l-th transition
    chosen_config = new_configs[l]
    i, j, new_i, new_j = chosen_config[0], chosen_config[1], chosen_config[2], chosen_config[3]
    lattice = update_lattice(lattice = lattice, i = i, j = j, new_i = new_i, new_j = new_j)
    
    return lattice

In [9]:
N = 6
partnum = 4
Beta = 1
M = create_lattice(N = N, particle_number = partnum)
M

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.]])

In [10]:
newconfigs, rates = new_configurations(lattice = M, N = N, Beta = Beta)

In [11]:
newM = find_transition(lattice = M, N = N, new_configs = newconfigs, rates = rates, Beta = Beta)

In [12]:
newM

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [13]:
def correlations(lattice):
    neighs = create_neighs()
    
    corr = 0
    
    N = len(lattice)
    
    for i in range(N):
        for j in range(N):
            for neigh in neighs[i%2]: #iterate through the neighbours
                neigh_i, neigh_j = (i+neigh[0])%N, (j+neigh[1])%N #position of the neighbour 
                corr += lattice[i,j]*lattice[neigh_i,neigh_j] #add the correlations 
                
    return corr/2 #avoid double counting

In [14]:
def simulate_hoppings(N, particle_number, Beta,it_num):
    lattice = create_lattice(N = N, particle_number = particle_number)
    corrs = []
    corrs.append(correlations(lattice))
    #do it_num iterations
    for n in range(it_num):
        #create a new configuration with transition rates
        newconfigs, rates = new_configurations(lattice = lattice, N = N, Beta = Beta)
        
        #update the lattice according to the transition rates
        lattice = find_transition(lattice = lattice, N = N, new_configs = newconfigs, rates = rates, Beta = Beta)
        
        #add the new correlations to the corrs list
        corrs.append(correlations(lattice))

    return np.array(corrs)

In [15]:
import time

In [16]:
N = 40
particle_number = 800
Beta = 0.1

In [20]:
%%time
it_num = 1000
cors = simulate_hoppings(N = N, particle_number = particle_number, Beta = Beta, it_num = it_num)

CPU times: user 18.3 s, sys: 0 ns, total: 18.3 s
Wall time: 18.4 s


In [ ]:
plt.plot(cors)
plt.xlabel("Iteration",fontsize = 14)
plt.ylabel("Correlation", fontsize = 14)
plt.show()

In [ ]:
cors[-1]/particle_number

In [ ]:
N = 40
particle_number = 400
Beta = 0.8

In [ ]:
it_num = 5000
cors = simulate_hoppings(N = N, particle_number = particle_number, Beta = Beta, it_num = it_num)

In [ ]:
plt.plot(cors)
plt.xlabel("Iteration",fontsize = 14)
plt.ylabel("Correlation", fontsize = 14)
plt.show()

In [ ]:
cors[-1]/particle_number

In [ ]:
def simulate_hoppings(N, particle_number, Betai, Betaf ,it_num):
    lattice = create_lattice(N = N, particle_number = particle_number)
    corrs = []
    corrs.append(correlations(lattice))
    
    #linear cooling of the sample from Betai to Betaf
    m = (Betaf-Betai)/(it_num+1)
    Beta = Betai
    #do it_num iterations
    for n in range(it_num):
        #first cool down the sample a bit
        Beta += m
        
        #create a new configuration with transition rates
        newconfigs, rates = new_configurations(lattice = lattice, N = N, Beta = Beta)
        
        #update the lattice according to the transition rates
        lattice = find_transition(lattice = lattice, N = N, new_configs = newconfigs, rates = rates, Beta = Beta)
        
        #add the new correlations to the corrs list
        corrs.append(correlations(lattice))

    return np.array(corrs)

In [ ]:
N = 40
particle_number = 400
Betai = 0.5
Betaf = 1.2
it_num = 20000

In [ ]:
cors = simulate_hoppings(N = N, particle_number = particle_number, Betai = Betai, Betaf = Betaf, it_num = it_num)

In [ ]:
plt.plot(cors)
plt.xlabel("Iteration",fontsize = 14)
plt.ylabel("Correlation", fontsize = 14)
plt.show()

In [ ]:
cors[-1]/particle_number